In [3]:
path = "../data/btcusdt/20231030/take_1/orderbook/"

In [4]:
import json
import os

files = os.listdir(path)
files.sort()

files

['210448164521_ob.json', '210454944953.json', '210501392266_ob.json']

In [5]:
orderbook_path = f"{path}{files[0]}"
f = open(orderbook_path)
orderbook_raw = json.load(f)
orderbook = {
    "bids" : {float(b[0]) : float(b[1]) for b in orderbook_raw["bids"]},
    "asks" : {float(a[0]) : float(a[1]) for a in orderbook_raw["asks"]}
}

orderbook

{'bids': {34363.01: 10.89581,
  34362.45: 0.0007,
  34362.01: 0.0007,
  34362.0: 0.14294,
  34361.99: 0.00585,
  34361.96: 0.21314,
  34361.8: 0.2,
  34361.77: 0.00134,
  34361.68: 0.14555,
  34361.57: 0.0007},
 'asks': {34363.02: 1.40375,
  34363.29: 0.00134,
  34363.31: 0.00019,
  34363.33: 0.14889,
  34363.52: 0.00224,
  34363.65: 0.00019,
  34363.69: 0.0002,
  34363.77: 0.0007,
  34363.99: 0.00019,
  34364.05: 0.00134}}

In [13]:
orderbook_path = f"{path}{files[0]}"
f = open(orderbook_path)
orderbook_raw = json.load(f)
orderbook = {
    "bids" : {float(b[0]) : float(b[1]) for b in orderbook_raw["bids"]},
    "asks" : {float(a[0]) : float(a[1]) for a in orderbook_raw["asks"]}
}

updates_path = f"{path}{files[1]}"
f = open(updates_path)
updates_raw = json.load(f)

for u in updates_raw:

    #u = updates_raw[0]

    update = {
    "bids" : {float(b[0]) : float(b[1]) for b in u["b"]},
    "asks" : {float(a[0]) : float(a[1]) for a in u["a"]}
    }

    for b in update["bids"]:
        if (update["bids"][b] != 0.0):
            orderbook["bids"][b] = update["bids"][b]
        elif (b in orderbook["bids"]):
            del orderbook["bids"][b]
        else:
            pass

In [15]:
len(orderbook["bids"])

213

In [16]:
orderbook["bids"]

{34363.01: 0.2579,
 34362.45: 0.0007,
 34362.01: 0.0007,
 34362.0: 0.14294,
 34361.96: 0.19998,
 34361.8: 0.2,
 34361.77: 0.00134,
 34361.68: 0.02472,
 34361.57: 0.0007,
 34361.24: 0.00224,
 34361.13: 0.0007,
 34361.01: 0.04077,
 34360.48: 0.00224,
 34360.16: 0.20098,
 34358.74: 0.23736,
 34357.22: 0.43576,
 34340.01: 0.61195,
 34321.99: 0.25,
 29880.87: 0.04956,
 10000.0: 119.59789,
 34360.69: 0.0007,
 34357.38: 0.63422,
 34354.05: 0.04833,
 34351.53: 0.43576,
 34349.83: 0.4842,
 34346.13: 0.00029,
 34340.2: 0.00021,
 34328.14: 0.0003,
 30926.0: 0.20391,
 34362.84: 0.14498,
 34357.07: 0.46294,
 34278.81: 0.1,
 34240.05: 0.00021,
 34227.0: 0.00021,
 34213.95: 0.00021,
 34194.6: 0.00016,
 34187.84: 0.00021,
 34359.27: 0.125,
 34351.77: 0.26295,
 34341.49: 0.66237,
 34323.69: 0.34955,
 34055.0: 1.63729,
 34364.0: 0.1407,
 34363.99: 0.20017,
 34363.32: 0.001,
 34363.13: 0.2,
 34363.02: 0.2,
 34360.85: 0.19998,
 34360.24: 0.2,
 34359.55: 0.00639,
 34356.16: 0.00073,
 34355.61: 0.28911,
 34

In [17]:
import collections

od = collections.OrderedDict(sorted(orderbook["bids"].items()))

od

OrderedDict([(10000.0, 119.59789),
             (17183.52, 0.01644),
             (29871.11, 0.07079),
             (29880.87, 0.04956),
             (30926.0, 0.20391),
             (30930.0, 0.27227),
             (34003.75, 0.03083),
             (34055.0, 1.63729),
             (34109.21, 0.00015),
             (34187.84, 0.00021),
             (34194.6, 0.00016),
             (34205.29, 0.00021),
             (34213.05, 0.00021),
             (34213.95, 0.00021),
             (34227.0, 0.00021),
             (34228.86, 0.24006),
             (34233.02, 0.00651),
             (34240.05, 0.00021),
             (34250.08, 0.28373),
             (34259.09, 0.00021),
             (34265.96, 0.00073),
             (34275.0, 0.72198),
             (34278.81, 0.1),
             (34299.69, 0.00015),
             (34311.51, 0.00018),
             (34311.52, 0.02808),
             (34314.55, 3.12016),
             (34314.58, 1.85042),
             (34316.26, 0.5876),
             (34316.66, 